In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc4
from matplotlib.gridspec import GridSpec

import pandas as pd

In [22]:
def read_csv(directory):

    df = pd.DataFrame()

    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            temp_df = pd.read_csv(filepath)
            df = pd.concat([df, temp_df])

    df['mean'] = df['mean'].replace('--', np.nan)
    df['region'] = df['region'].astype('category')
    df['datetime'] = pd.to_datetime(df['time'], unit='D', origin='1970-01-01')
    df['year'] = df['datetime'].dt.year
    df['dec_yr'] = df['datetime'].dt.year + (df['datetime'].dt.dayofyear - 1) / 365
    df['mean'] = df['mean'].astype(float)

    return df

def get_timeseries(df):
        
    all_timeseries = []
    time = df['dec_yr'].unique()
    group = df.groupby('region')
    for name, group in group:
        all_timeseries.append(np.column_stack([group['dec_yr'], group['mean']]))

    return all_timeseries


In [23]:
def plot_chl_timeseries(project_folder, all_timeseries):

    fig = plt.figure(figsize=(8,12))
    plt.style.use('dark_background')
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    gs = GridSpec(6, 1, left=0.1, right=0.95, bottom=0.05, top=0.95)

    mean_spring_bloom = 2
    mean_summer_bloom = 1
    mean_peak_summer_bloom = 2.2

    for region in range(6):
        ax1 = fig.add_subplot(gs[5-region, 0])

        for month in range(12):
            ax1.plot([month/12, month/12],[0,6],'w--', linewidth=0.5, alpha=0.5)
        ax1.plot([0, 1], [2, 2], 'w--', linewidth=0.5, alpha=0.5)
        ax1.plot([0, 1], [4, 4], 'w--', linewidth=0.5, alpha=0.5)
        ax1.plot([0, 1], [1, 1], 'w--', linewidth=0.5, alpha=0.5)

        year_timeseries = all_timeseries[region]
        for year in range(1992,2022):
            subset_timeseries = year_timeseries[np.logical_and(year_timeseries[:,0]>=year,
                                                               year_timeseries[:,0]<year+1), :]
            subset_timeseries[:,0] -= year
            if len(subset_timeseries)>0:
                ax1.plot(subset_timeseries[:,0], subset_timeseries[:,1], color = colors[6], linewidth=0.8)
        ax1.text(0.02, 5.8,'Region '+str(region+1), va='top', ha='left')
        ax1.set_xlim([0,1])
        ax1.set_ylim([0,6])

        ax1.set_xticks(np.arange(1/24,1,1/12))
        # if region!=0:
        #     ax1.set_xticklabels([])
        # else:
        ax1.set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])
        if region==5:
            ax1.set_title('Chlorophyll Concentration in Arrigo et al 2017 Sample Areas')

        if region==3:
            ax1.set_ylabel('Chlorophyll Concentration (mg/cm$^3$)')

        ax1.plot([(31+28+31+7)/365, (31+28+31+7)/365], [0,6],'-', color = colors[4])
        ax1.plot([(31 + 28 + 31 + 30 + 31 + 30 + 7) / 365, (31 + 28 + 31 + 30 + 31 + 30 + 7) / 365], [0, 6], '-', color=colors[4])
        if region==5:
            ax1.text((31+28+31+4)/365, 5, 'Start of Spring Bloom $\\rightarrow$\nin Arrigo et al 2017     ',
                     ha='right', va='top', color = colors[4])
            ax1.text((31 + 28 + 31 + 30 + 31 + 30 + 10) / 365, 5, '$\leftarrow$ Start of Summer Bloom\n     in Arrigo et al 2017',
                     ha='left', va='top', color=colors[4])

        if region==0:
            ax1.text(0.6, 2, 'Mean Summer Chl.: 0.98 mg/cm$^3$')
            ax1.text(0.6, 3.5, 'Mean Peak Summer Chl.: 2.2 mg/cm$^3$')


    plt.savefig(os.path.join(project_folder,'figures','Chlorophyll in Arrigo et al Areas.png'))
    plt.close(fig)

In [24]:
df = read_csv('../means/')
all_timeseries = get_timeseries(df)
del df

project_folder = '..'

plot_chl_timeseries(project_folder, all_timeseries)